In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# States

In [2]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

print pop.head()
print areas.head()
print abbrevs.head()

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [3]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [4]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [5]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [6]:
merged['state/region'][merged['state'].isnull()].unique()

array(['PR', 'USA'], dtype=object)

In [7]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [8]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [9]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [10]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [11]:
final.dropna(inplace=True)
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [12]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [13]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [14]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

# Planets

In [17]:
planets = sns.load_dataset('planets')
print planets.shape
print planets.head()
planets.describe()

(1035, 6)
            method  number  orbital_period   mass  distance  year
0  Radial Velocity       1         269.300   7.10     77.40  2006
1  Radial Velocity       1         874.774   2.21     56.95  2008
2  Radial Velocity       1         763.000   2.60     19.84  2011
3  Radial Velocity       1         326.030  19.40    110.62  2007
4  Radial Velocity       1         516.220  10.50    119.47  2009


,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [19]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [23]:
for (method, group) in planets.groupby('method'):
    print "{0:30s} shape={1}".format(method, group.shape)

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [26]:
planets.groupby('method').describe()

distance        mass      number  \
method                                                                   
Astrometry                  count     2.000000    0.000000    2.000000   
                            mean     17.875000         NaN    1.000000   
                            std       4.094148         NaN    0.000000   
                            min      14.980000         NaN    1.000000   
                            25%      16.427500         NaN    1.000000   
                            50%      17.875000         NaN    1.000000   
                            75%      19.322500         NaN    1.000000   
                            max      20.770000         NaN    1.000000   
Eclipse Timing Variations   count     4.000000    2.000000    9.000000   
                            mean    315.360000    5.125000    1.666667   
                            std     213.203907    1.308148    0.500000   
                            min     130.720000    4.200000    1.000000   
                            25%     130.720000    4.662500    1.000000   
                            50%     315.360000    5.125000    2.000000   
                            75%     500.000000    5.587500    2.000000   
                            max     500.000000    6.050000    2.000000   
Imaging                     count    32.000000    0.000000   38.000000   
                            mean     67.715937         NaN    1.315789   
                            std      53.736817         NaN    0.933035   
                            min       7.690000         NaN    1.000000   
                            25%      22.145000         NaN    1.000000   
                            50%      40.395000         NaN    1.000000   
                            75%     132.697500         NaN    1.000000   
                            max     165.000000         NaN    4.000000   
Microlensing                count    10.000000    0.000000   23.000000   
                            mean   4144.000000         NaN    1.173913   
                            std    2076.611556         NaN    0.387553   
                            min    1760.000000         NaN    1.000000   
                            25%    2627.500000         NaN    1.000000   
                            50%    3840.000000         NaN    1.000000   
...                                        ...         ...         ...   
Pulsation Timing Variations std            NaN         NaN         NaN   
                            min            NaN         NaN    1.000000   
                            25%            NaN         NaN    1.000000   
                            50%            NaN         NaN    1.000000   
                            75%            NaN         NaN    1.000000   
                            max            NaN         NaN    1.000000   
Radial Velocity             count   530.000000  510.000000  553.000000   
                            mean     51.600208    2.630699    1.721519   
                            std      45.559381    3.825883    1.157141   
                            min       1.350000    0.003600    1.000000   
                            25%      24.412500    0.225250    1.000000   
                            50%      40.445000    1.260000    1.000000   
                            75%      59.217500    3.000000    2.000000   
                            max     354.000000   25.000000    6.000000   
Transit                     count   224.000000    1.000000  397.000000   
                            mean    599.298080    1.470000    1.954660   
                            std     913.876990         NaN    1.399119   
                            min      38.000000    1.470000    1.000000   
                            25%     200.000000    1.470000    1.000000   
                            50%     341.000000    1.470000    1.000000   
                            75%     650.000000    1.470000    2.000000   
                            max    8500.00

In [27]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack()

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,NaN,NaN,NaN,2.0
Eclipse Timing Variations,NaN,NaN,5.0,10.0
Imaging,NaN,NaN,29.0,21.0
Microlensing,NaN,NaN,12.0,15.0
Orbital Brightness Modulation,NaN,NaN,NaN,5.0
Pulsar Timing,NaN,9.0,1.0,1.0
Pulsation Timing Variations,NaN,NaN,1.0,NaN
Radial Velocity,1.0,52.0,475.0,424.0
Transit,NaN,NaN,64.0,712.0
